### [DABP Final Project]
### The Expansion of the Pit Stop Program in San Francisco

# Generating Grids on the Map

This Jupyter notebook includes codes for generating grids on the map. This code is used to visualize our model outcomes on the map with color scales.

* we borrowed code from https://www.jpytr.com/post/analysinggeographicdatawithfolium/ and modified it to our need. 

In [ ]:
# import relevant libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster

In [ ]:
def get_geojson_grid(upper_right, lower_left, nlat=6, nlong=5):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], nlat+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], nlong+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [ ]:
# set coordinate boundaries of SF
lower_left = [37.708448, -122.514731]
upper_right = [37.811151, -122.357476]

In [ ]:
# get coordinates of center of SF
sf_lat = np.mean([37.708448, 37.811151])
sf_long = np.mean([-122.514731, -122.357476])

In [ ]:
# generate empty grid
m = folium.Map([sf_lat, sf_long], zoom_start = 12.2, tiles='cartodbpositron')
grid = get_geojson_grid(upper_right, lower_left , nlat=20, nlong=30)

for i, geo_json in enumerate(grid):

    color = 'White'

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.7,
                                                                        'dashArray': '1, 1',
                                                                        'fillOpacity': 0.1,
                                                                    })

    m.add_child(gj)
m

In [ ]:
# manipulate numpy array to display on the grid
result = np.random.randint(0, 5, (20,30))
result_flip = np.flip(result, axis=0)
result_flip_flat = result_flip.flatten()

In [ ]:
# create grid with colors 
m = folium.Map([sf_lat, sf_long], zoom_start = 12, tiles='cartodbpositron')

grid = get_geojson_grid(upper_right, lower_left, nlat=20, nlong=30)

# Add GeoJson to map
for i, box in enumerate(grid):
    geo_json = json.dumps(box)

    color = plt.cm.Reds(result_flip_flat[i] / 5)
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
    m.add_child(gj)

m